Convert the janky Google survey data into 3 normalized tables:
- People
    - ID, Name
- Proposition Categories
    - ID, Name
- Propositions
    - ID, Category ID, Text, Year
- Forecasts
    - ID, Proposition ID, Person ID, Probability

In [1]:
import pandas as pd

In [2]:
raw = pd.read_csv('raw_probabilities.csv')
raw.head()

,Timestamp,What's your name?,Can I track your answers on my website alongside others'?,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,...,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded ""GPT-5"" or ""ChatGPT-5"".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
0,2023/12/12 9:19:33 AM CST,Raluca Pavel,Yes,0.26,0.15,0.000,1.00,1.00,0.90,1.00,...,1.00,0.8,0.95,0.45,0.45,0.45,1.00,1.00,1.0,0.87
1,2023/12/12 11:18:08 AM CST,Ethan Swan,Yes,0.03,0.65,0.100,0.98,0.90,0.65,0.85,...,0.40,0.8,0.65,0.10,0.05,0.15,0.85,0.45,0.9,0.93
2,2023/12/12 9:44:25 PM CST,Tommy Callan,Yes,0.00,0.10,0.000,1.00,1.00,0.20,1.00,...,0.25,1.0,0.10,0.30,0.30,0.30,1.00,0.50,0.9,0.90
3,2023/12/13 5:03:40 PM CST,Greg Moore,Yes,0.01,0.05,0.000,0.95,0.95,0.90,0.90,...,0.10,0.9,0.87,0.15,0.15,0.15,0.40,0.95,0.7,0.70
4,2023/12/16 3:33:26 PM CST,Abhisek Gupta,Yes,0.01,0.50,0.001,0.90,0.75,0.10,0.90,...,0.01,0.5,0.70,0.10,0.10,0.10,0.95,0.70,0.8,0.70


In [3]:
people_names = raw["What's your name?"].str.strip().unique()
people_names

array(['Raluca Pavel', 'Ethan Swan', 'Tommy Callan', 'Greg Moore',
       'Abhisek Gupta', 'Fernando Garcia', 'Alex Van Fossen',
       'David Schmitz', 'Phil Anderson', 'Brian Van Fossen', 'Lizz Hyde'],
      dtype=object)

In [4]:
user_df = pd.DataFrame({'id': range(len(people_names)), 'name': people_names})
user_df['email'] = ''
user_df

,id,name,email
0,0,Raluca Pavel,
1,1,Ethan Swan,
2,2,Tommy Callan,
3,3,Greg Moore,
4,4,Abhisek Gupta,
5,5,Fernando Garcia,
6,6,Alex Van Fossen,
7,7,David Schmitz,
8,8,Phil Anderson,
9,9,Brian Van Fossen,


In [5]:
users_df = pd.DataFrame({'id': range(len(people_names)), 'name': people_names})
users_df['email'] = ''
users_df

,id,name,email
0,0,Raluca Pavel,
1,1,Ethan Swan,
2,2,Tommy Callan,
3,3,Greg Moore,
4,4,Abhisek Gupta,
5,5,Fernando Garcia,
6,6,Alex Van Fossen,
7,7,David Schmitz,
8,8,Phil Anderson,
9,9,Brian Van Fossen,


In [6]:
nonprop_columns = ("What's your name?", "Timestamp", "Can I track your answers on my website alongside others'?")
props = [col for col in raw.columns if col not in nonprop_columns]
props[:4], props[-4:]

(['Geopolitics | China lands ground forces on the island of Taiwan',
  'Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year',
  'Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.',
  'Geopolitics | Xi Jinping is still the premier of China at the end of 2024.'],
 ['Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.',
  'Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.',
  'Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.',
  'Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.'])

In [7]:
categories = [prop.split(' | ')[0] for prop in props]
uniq_categories = set(categories)

category_df = pd.DataFrame({'id': range(len(uniq_categories)), 'name': list(uniq_categories)})
category_df

,id,name
0,0,US Politics
1,1,Geopolitics
2,2,Business & Tech
3,3,Sports & Culture


In [8]:
prop_names = [prop.split(' | ')[1] for prop in props]

category_ids = map(lambda cat_name: category_df[category_df['name'] == cat_name].iloc[0]['id'], categories)
prop_df = pd.DataFrame({
    'id': range(len(prop_names)),
    'text': prop_names,
    'category_id': category_ids,
    'year': 2024,
})
prop_df.sample(5)

,id,text,category_id,year
13,13,At least one third-party US presidential candi...,0,2024
14,14,The US president elect dies between the electi...,0,2024
25,25,Lionel Messi retires from or otherwise leaves ...,3,2024
12,12,A Democrat candidate who isn't Joe Biden is el...,0,2024
4,4,Narendra Modi is still the prime minister of I...,1,2024


Loop over the individual forecasts and build rows out of them.

In [9]:
forecast_rows = []
next_id = 0
for _, row in raw.iterrows():
    person = row["What's your name?"].strip()
    user_id = users_df[users_df['name'] == person].iloc[0]['id']
    user_id = int(user_id)
    for prop in props:
        prop_name = prop.split(' | ')[1]
        prop_id = prop_df[prop_df['text'] == prop_name].iloc[0]['id']
        prop_id = int(prop_id)
        prop_forecast = float(row[prop])
        forecast_rows.append((next_id, prop_id, user_id, prop_forecast))
        next_id += 1

forecast_df = pd.DataFrame(forecast_rows, columns=['id', 'prop_id', 'user_id', 'forecast'])
forecast_df

,id,prop_id,user_id,forecast
0,0,0,0,0.26
1,1,1,0,0.15
2,2,2,0,0.00
3,3,3,0,1.00
4,4,4,0,1.00
...,...,...,...,...
435,435,35,10,0.85
436,436,36,10,0.90
437,437,37,10,0.75
438,438,38,10,0.80


First, save all the data to CSVs for redundancy.

In [10]:
users_df.to_csv('data/users.csv', index=False)
category_df.to_csv('data/categories.csv', index=False)
prop_df.to_csv('data/props.csv', index=False)
forecast_df.to_csv('data/forecasts.csv', index=False)

Then push all these tables to the DB.

In [11]:
import dotenv

dotenv.load_dotenv()

True

Prepare everything for the record insertion.

In [12]:
import os
import sqlalchemy as sa

conn_str = os.environ['DATABASE_URL']
conn_str = conn_str.replace('postgresql://', 'postgresql+psycopg://')

engine = sa.create_engine(conn_str)

metadata = sa.MetaData()

# Create the tables with the correct schema
# We could let pandas create the tables but it's more reliable to do it manually.
users_table = sa.Table(
    'users',
    metadata,
    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
    sa.Column('name', sa.String),
    sa.Column('email', sa.String),
)
categories_table = sa.Table(
    'categories',
    metadata,
    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
    sa.Column('name', sa.String),
)
props_table = sa.Table(
    'props',
    metadata,
    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
    sa.Column('text', sa.String),
    sa.Column('category_id', sa.Integer, sa.ForeignKey('categories.id')),
    sa.Column('year', sa.Integer),
)
forecasts_table = sa.Table(
    'forecasts',
    metadata,
    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
    sa.Column('prop_id', sa.Integer, sa.ForeignKey('props.id')),
    sa.Column('user_id', sa.Integer, sa.ForeignKey('users.id')),
    sa.Column('forecast', sa.Float),
)
# We don't have records for this table yet, but we'll add it later.
resolutions_table = sa.Table(
    'resolutions',
    metadata,
    sa.Column('id', sa.Integer, primary_key=True, autoincrement=True),
    sa.Column('prop_id', sa.Integer, sa.ForeignKey('props.id')),
    sa.Column('resolution', sa.Boolean),
)

Check that the tables don't exist yet, then create and load them.

In [13]:
inspector = sa.inspect(engine)
if inspector.has_table('users'):
    raise Exception('Table users already exists')
if inspector.has_table('categories'):
    raise Exception('Table categories already exists')
if inspector.has_table('props'):
    raise Exception('Table props already exists')
if inspector.has_table('forecasts'):
    raise Exception('Table forecasts already exists')
if inspector.has_table('resolutions'):
    raise Exception('Table resolutions already exists')

metadata.create_all(engine)

user_df.to_sql('users', engine, index=False, if_exists='append')
category_df.to_sql('categories', engine, index=False, if_exists='append')
prop_df.to_sql('props', engine, index=False, if_exists='append')
forecast_df.to_sql('forecasts', engine, index=False, if_exists='append')

-1

Last, create a view that merges everything into a single table.

In [17]:
view_sql = '''
CREATE VIEW v_forecasts AS
SELECT
    u.id as user_id,
    u.name as user_name,
    c.id as category_id,
    c.name as category_name,
    p.id as prop_id,
    p.text as prop_text,
    f.forecast,
    r.resolution,
    POWER(r.resolution::INTEGER - f.forecast, 2) AS score
FROM users u
INNER JOIN forecasts f
    ON f.user_id = u.id
INNER JOIN props p
    ON p.id = f.prop_id
INNER JOIN categories c
    ON c.id = p.category_id
LEFT JOIN resolutions r
    ON p.id = r.prop_id
-- Inner join conditions
'''

with engine.connect() as conn:
    conn.execute(sa.text(view_sql))
    conn.commit()